In [1]:
import pymysql
import requests

### 下載分析pm2.5 opendata資料

In [6]:
url="https://data.moenv.gov.tw/api/v2/aqx_p_02?api_key=540e2ca4-41e1-4186-8497-fdd67024ac44&limit=1000&sort=datacreationdate%20desc&format=JSON"
url

'https://data.moenv.gov.tw/api/v2/aqx_p_02?api_key=540e2ca4-41e1-4186-8497-fdd67024ac44&limit=1000&sort=datacreationdate%20desc&format=JSON'

In [7]:
resp=requests.get(url,verify=False)
resp

c:\ProgramData\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.moenv.gov.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>

In [8]:
datas=resp.json()['records']
datas

[{'site': '員林',
  'county': '彰化縣',
  'pm25': '14',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '大城',
  'county': '彰化縣',
  'pm25': '',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '富貴角',
  'county': '新北市',
  'pm25': '4',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '麥寮',
  'county': '雲林縣',
  'pm25': '11',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '關山',
  'county': '臺東縣',
  'pm25': '3',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '馬公',
  'county': '澎湖縣',
  'pm25': '7',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '金門',
  'county': '金門縣',
  'pm25': '6',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '馬祖',
  'county': '連江縣',
  'pm25': '',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '埔里',
  'county': '南投縣',
  'pm25': '10',
  'datacreationdate'

In [9]:
len(datas)

1000

In [10]:
datas[0]

{'site': '員林',
 'county': '彰化縣',
 'pm25': '14',
 'datacreationdate': '2025-08-27 20:00',
 'itemunit': 'μg/m3'}

### 建立資料表

In [3]:
table_str="""
create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
county varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20),
unique key site_time (site,datacreationdate)
)
"""
print(table_str)


create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
county varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20),
unique key site_time (site,datacreationdate)
)



### 連接資料庫

In [2]:
# conn=pymysql.connect(
# host="127.0.0.1",
# user="root",
# password="",
# port=3307,
# database="demo"
# )

conn=pymysql.connect(
host="mysql-3597b72c-ouhanxiang484-pm25project.i.aivencloud.com",
user="avnadmin",
password="AVNS_niEr1CxTASTTpMrZ-5P",
port=12799,
database="defaultdb"
)
cursor=conn.cursor()
cursor


In [4]:
cursor.execute(table_str)
conn.commit()

### 寫入一筆資料

In [11]:
sqlstr="insert into pm25(site,county,pm25,datacreationdate,itemunit)\
      values('{}','{}',{},'{}','{}')"

data=list(datas[0].values())
print(data)
sqlstr.format(data[0],data[1],data[2],data[3],data[4])

['員林', '彰化縣', '14', '2025-08-27 20:00', 'μg/m3']


"insert into pm25(site,county,pm25,datacreationdate,itemunit)      values('員林','彰化縣',14,'2025-08-27 20:00','μg/m3')"

In [ ]:
cursor.execute(sqlstr.format(data[0],data[1],data[2],data[3],data[4]))
conn.commit()

### 一次寫入  
- 加上 ignore

In [12]:
sqlstr="insert ignore into pm25(site,county,pm25,datacreationdate,itemunit)\
      values(%s,%s,%s,%s,%s)"

# 移除pm25空字串部分
values=[list(data.values()) for data in datas if list(data.values())[2]!=""]

cursor.executemany(sqlstr,values)
conn.commit()

In [13]:
conn.close()